In [1]:
# importing libraries 
import cv2 
import numpy as np 

In [2]:
# Create a VideoCapture object and read from input file 
cap = cv2.VideoCapture('Clips\Optical flow_1.mov') 

<>:2: SyntaxWarning: invalid escape sequence '\O'
<>:2: SyntaxWarning: invalid escape sequence '\O'
C:\Users\Jakrin PC\AppData\Local\Temp\ipykernel_32352\2540164318.py:2: SyntaxWarning: invalid escape sequence '\O'
  cap = cv2.VideoCapture('Clips\Optical flow_1.mov')


In [3]:
# Check if camera opened successfully 
if (cap.isOpened()== False): 
    print("Error opening video file")

In [4]:
kernelSharpness = np.array([[0,-1,0],[-1,5,-1],[0,-1,0],]) #sharpening
kernelcustom = np.array([[0,-2,0],[-2,8,-2],[0,-2,0],])    #Edge detection
Wc = 700
Hc = 1000       #Cropping resolution (700x1000)
a = Wc/2
b = Hc/2    
xtl = 190
ytl = 0         #the top-left of the cropping section
x = xtl + a
y = ytl + b     #Center of result video output
d = 0           #Order of license plate

In [5]:
# Read until video is completed 
while(cap.isOpened()): 
    
# Capture frame-by-frame 
    ret, frame = cap.read() 
    
    if ret == True:         
        frameEdit = cv2.getRectSubPix(frame,(Wc,Hc),(x,y)) #Making Black-white cropped video (frameEdit)
        frame = cv2.getRectSubPix(frame,(Wc,Hc),(x,y))     #Making Color cropped video (frame)
        
        frame = cv2.filter2D(frame,0,kernelSharpness)      #Add sharpness in "frame"
        frameEdit = cv2.filter2D(frameEdit,0,kernelcustom) #Add edge detection in "frameEdit"

        frameEdit = cv2.cvtColor(frameEdit,cv2.COLOR_RGB2GRAY)              #Change "frameEdit" to grayscale video
        ret, frameEdit = cv2.threshold(frameEdit,5,255,cv2.THRESH_BINARY)   #Partition the license plate from the hull in "frameEdit"

        contours, _= cv2.findContours(frameEdit, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE) 

        frameEdit = cv2.cvtColor(frameEdit,cv2.COLOR_GRAY2RGB) #Change "frameEdit" to RGB video

        for i in range(len(contours)): #Run for every contour in current image
            xb,yb,wb,hb = cv2.boundingRect(contours[i]) #determine the position and scale of each contour
            aRatio =wb/hb #Ration of the width and height
            area =wb*hb   #Area
            if (5000<area<15000 and 2.3<aRatio<2.8): #Pick the contour that have features similar to license plates
                originalPoints = [(xb,yb),(xb+wb,yb),(xb,yb+hb),(xb+wb,yb+hb)] #Determine perspective transfromation for cropping license plate
                targetPoints = [(0,0),(135,0),(0,60),(135,60)]                 #Determine the license plate scale (≈ 13.5 in /5.9 in)

                licensePlate = cv2.getPerspectiveTransform(np.float32(originalPoints),np.float32(targetPoints))
                licensePlate = cv2.warpPerspective(frame,licensePlate,(135,60))
                cv2.imshow('License plate', licensePlate)                                                           #Use perspective transfromation for cropping license plate
                licensePlateBW = cv2.getPerspectiveTransform(np.float32(originalPoints),np.float32(targetPoints))
                licensePlateBW = cv2.warpPerspective(frameEdit,licensePlateBW,(135,60))
                cv2.imshow('License plate Black-White', licensePlateBW)                                             #Use perspective transfromation for cropping license plate
 
                cv2.putText(frame, f'Area :{area}', (xb, yb-50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)         #Show area of the contour
                cv2.putText(frame, f'Ratio :{aRatio:.2f}', (xb, yb-20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)  #Show ratio of the contour's width and height

                cv2.drawContours(frameEdit, contours[i], -1, (0,0,255), 2)              #Draw the red line for the real contour on the greyscale video
                cv2.rectangle(frame,(xb,yb),(xb+wb,yb+hb),(0,255,0),thickness =1)       #Draw the green rectangle on the RGB video
                cv2.rectangle(frameEdit,(xb,yb),(xb+wb,yb+hb),(255,0,0),thickness =1)   #Draw the Blue rectangle on the greyscale video

                cv2.imwrite('LicensePlates/img%d.png'%d, licensePlate)  #Save the license plate image on determined path
                d+=1

        cv2.imshow('Frame from source', frame)  
        cv2.imshow('Frame from edit', frameEdit)   #Show every frame
            
    # Press Q on keyboard to exit 
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
  
# Break the loop 
    else: 
        break
  
# When everything done, release the video capture object 
cap.release() 
  
# Closes all the frames 
cv2.destroyAllWindows() 